In [11]:
import os
import re
import sys
import numpy as np
import pandas as pd
import nibabel as nib
import subprocess

In [9]:
# Define global variable(s)
# scripts_dir = os.path.dirname(os.path.realpath(__file__))
scripts_dir = os.getcwd()

In [2]:
class Command():
    '''
    Creates a command and an empty command list for UNIX command line programs/applications. Primary use and
    use-cases are intended for the subprocess module and its associated classes (i.e. call/run).
    Attributes:
        command: Command to be performed on the command line
    '''

    def __init__(self):
        '''
        Init doc-string for Command class.
        '''
        pass

    def init_cmd(self, command):
        '''
        Init command function for initializing commands to be used on UNIX command line.
        
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        return self.cmd_list

In [3]:
def run(cmd_list,stdout="",stderr=""):
    '''
    Uses python's built-in subprocess class to run a command from an input command list.
    The standard output and error can optionally be written to file.
    
    Arguments:
        cmd_list(list): Input command list to be run from the UNIX command line.
        stdout(file): Output file to write standard output to.
        stderr(file): Output file to write standard error to.
    Returns:
        stdout(file): Output file that contains the standard output.
        stderr(file): Output file that contains the standard error.
    '''
    if stdout and stderr:
        with open(stdout,"w") as file:
            with open(stderr,"w") as file_err:
                subprocess.call(cmd_list,stdout=file,stderr=file_err)
                file.close(); file_err.close()
    elif stdout:
        with open(stdout,"w") as file:
            subprocess.call(cmd_list,stdout=file)
            file.close()
        stderr = None
    else:
        subprocess.call(cmd_list)
        stdout = None
        stderr = None

    return stdout,stderr

In [28]:
def roi_loc(coords,vol_atlas_num=3):
    '''
    Uses input list of X,Y,Z MNI space mm coordinates to identify ROIs.
    
    NOTE: External bash script is used.
    
    Arguments:
        coords(list): Coordinate list with a lenth of 3 that corresponds to the XYZ coordinates of some ROI in MNI space.
        vol_atlas_num(int): Atlas to be used in FSL's `atlasquery`. Number corresponds to an atlas. See FSL's `atlasquery` help menu for details.
    Returns:
        roi_list(list): List of ROIs generated from input coordinates.
    '''
    
    # Define volume atlas number dictionary
    vol_atlas_dict = {
    1: "Cerebellar Atlas in MNI152 space after normalization with FLIRT",
    2: "Cerebellar Atlas in MNI152 space after normalization with FNIRT",
    3: "Harvard-Oxford Cortical Structural Atlas",
    4: "Harvard-Oxford Subcortical Structural Atlas",
    5: "Human Sensorimotor Tracts Labels",
    6: "JHU ICBM-DTI-81 White-Matter Labels",
    7: "JHU White-Matter Tractography Atlas",
    8: "Juelich Histological Atlas",
    9: "MNI Structural Atlas",
    10: "Mars Parietal connectivity-based parcellation",
    11: "Mars TPJ connectivity-based parcellation",
    12: "Neubert Ventral Frontal connectivity-based parcellation",
    13: "Oxford Thalamic Connectivity Probability Atlas",
    14: "Oxford-Imanova Striatal Connectivity Atlas 3 sub-regions",
    15: "Oxford-Imanova Striatal Connectivity Atlas 7 sub-regions",
    16: "Oxford-Imanova Striatal Structural Atlas",
    17: "Sallet Dorsal Frontal connectivity-based parcellation",
    18: "Subthalamic Nucleus Atlas",
    19: "Talairach Daemon Labels"}
    
    # Define list and output file
    roi_list = list()
    out_file = "subcort.rois.txt"
    
    if len(coords) == 3:
        atlasq_cmd = os.path.join(scripts_dir,"atlasq.sh")
        atlasq = Command().init_cmd(atlasq_cmd)
        atlasq.append(f"--coord")
        atlasq.append(f"\"{coords[0]},{coords[1]},{coords[2]}\"")
        atlasq.append("--atlas-num")
        atlasq.append(f"{vol_atlas_num}")
    
        run(atlasq,out_file)

        with open(out_file,"r") as file:
            text = file.readlines()
            for i in range(0,len(text)):
                text[i] = re.sub(f"<b>{vol_atlas_dict[vol_atlas_num]}</b><br>","",text[i].rstrip())

        os.remove(out_file)
        
        if len(text) == 0:
            pass
        else:
            roi_list.extend(text) 
        
    return roi_list

In [5]:
def vol_clust(nii_file,thresh=0.95,dist=0,vol_atlas_num=3):
    '''
    Identifies clusters in a volumetric (NIFTI) file.
    
    Arguments:
        nii_file(file): Input NIFTI file
        thresh(float): Cluster minimum threshold
        dist(float): Minimum distance between clusters
        vol_atlas_num(int): Atlas to be used in FSL's `atlasquery`. Number corresponds to an atlas. See FSL's `atlasquery` help menu for details.
    Returns:
        roi_list(list): List of ROIs that overlap with some given cluster
    '''
    
    out_file = "vol.cluster.tsv"
    
    roi_list = list()
    tmp_list = list()
    
    vol_clust = Command().init_cmd("cluster")
    
    vol_clust.append(f"--in={nii_file}")
    vol_clust.append(f"--thresh={thresh}")
    vol_clust.append(f"--peakdist={dist}")
    vol_clust.append("--mm")
    
    run(vol_clust,out_file)
    
    df_tmp = pd.read_csv(out_file,sep="\t")
    
    os.remove(out_file)
    
    df = df_tmp[['MAX X (mm)','MAX Y (mm)','MAX Z (mm)']].copy()
    
    for i in range(0,len(df)):
        coord_list=[df['MAX X (mm)'][i],df['MAX Y (mm)'][i],df['MAX Z (mm)'][i]]
        tmp_list = roi_loc(coord_list,vol_atlas_num)
        if len(tmp_list) == 0:
            pass
        else:
            roi_list.extend(tmp_list)
    
    return roi_list

In [6]:
n = "test.files/dr_stage3_ic0002_tfce_corrp_tstat2.nii.gz"

In [27]:
vol_clust(n,thresh=0.95,dist=0,vol_atlas_num=3)

['11% Cingulate Gyrus, posterior division']

In [25]:
roi_loc([24,-28,-10],4)

['73% Right Hippocampus, 14% Right Cerebral White Matter, 3% Right Cerebral Cortex']

In [ ]:
def proc_vol(nii_file,out_file,thresh = 0.95, dist = 0, vol_atlas_num = 3, nii_atlas = "", atlas_info = ""):
    '''
    Identifies ROIs that have overlap with some cluster(s) from the input NIFTI file.
    
    Arguments:
        nii_file(NIFTI file): Input NIFTI volume file
        out_file(file): Name for output CSV
        thresh(float): Threshold values below this value
        dist(float): Minimum distance between two or more clusters
        vol_atlas_num(int): Atlas to be used in FSL's `atlasquery`. Number corresponds to an atlas. See FSL's `atlasquery` help menu for details.
        nii_atlas(NIFTI file): NIFTI atlas file
        atlas_info(file): Corresponding CSV key, value pairs of ROIs for atlas file
    Returns:
      out_filefile(file): Output CSV file
    '''
    
    if nii_atlas and atlas_info:
        # Read atlas data and info
        [atlas_data,atlas_dict] = load_atlas_data(nii_atlas,atlas_info)

        # Read NIFTI data and find clusters
        img_data = load_nii_vol(nii_file,thresh,dist)

        # Identify cluster and ROI overlaps
        roi_list = get_roi_name(img_data,atlas_data,atlas_dict)
    else:
        roi_list = vol_clust(nii_file,thresh,dist,vol_atlas_num)
    
    # Write spreadsheet to file
    if len(roi_list) != 0:
        out_file = write_spread(nii_file,out_file,roi_list)
    
    return out_file